In [43]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams, get_associated_token_address
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [44]:
program_id = PublicKey("GWJQHiudaVnhBmvCkRmMPeWs2rffhG2fmkXVRJHM9t5s")
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
# program_id_account_meta

In [45]:
PublicKey.find_program_address([b"Governor_Vault"], program_id)

(4eeqV9wkdBYJyVopZiTkUoH5XE7JpYJfavZ4NkKbvWW6, 254)

In [46]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [47]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
payer_keypair.public_key

2ZHc9QxDDaJwqNEFzpAGUrvxCWZSNnXSffHxV9hG2axp

In [48]:
mint_account = Keypair.generate()

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
vault_pda, _ = PublicKey.find_program_address([b"Governor_Vault"], program_id)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

avatar_mint = PublicKey("8Z7vfkpxENuAV1joZZZDLmQmkUspgJuMhkLiUbq6jcjT")
avatar_program = PublicKey("DvTvXKZR9wveQzL3pBM7QJ9YZpConSXBXRTyDsEADNqX")

In [49]:
mint_account_meta = AccountMeta(mint_account.public_key, True, True)
associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, mint_account.public_key)
associated_account_meta = AccountMeta(associated_token_address, False, True)

(mint_authority, nonce) = PublicKey.find_program_address([b'mint_authority'], program_id)
(avatar_data_pda, avatar_pda_nonce) = PublicKey.find_program_address([b'governor_data_pda', bytes(mint_account_meta.pubkey)], program_id)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(mint_account_meta.pubkey)], metadata_program_pubkey)
(avatar_metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(avatar_mint)], metadata_program_pubkey)
avatar_creator , _ = PublicKey.find_program_address([b"avatar_mint_authority_pda"], avatar_program)
avatar_associated_account = instructions.get_associated_token_address(payer_keypair.public_key, avatar_mint)
vault_account_meta = AccountMeta(vault_pda, False, True)


avatar_data_pda_meta = AccountMeta(avatar_data_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
sales_pda_meta = AccountMeta(sales_pda, False, True)
mint_authority_meta = AccountMeta(mint_authority, False, False)

accounts = [
    payer_account_meta, 
    vault_account_meta, 
    mint_account_meta, 
    rent_account_meta, 
    associated_account_meta,
    spl_program_meta,
    AccountMeta(SYS_PROGRAM_ID, False, False),
    mint_authority_meta,
    avatar_data_pda_meta,
    sales_pda_meta,
    AccountMeta(metadata_pda, False, True),
    sysvar_clock_meta,
    AccountMeta(avatar_mint, False, False),
    AccountMeta(avatar_creator, False, False),
    AccountMeta(avatar_metadata_pda, False, False),
    AccountMeta(avatar_associated_account, False, False),

    spl_program_meta,
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    spl_program_meta,
    spl_program_meta,
    AccountMeta(metadata_program_pubkey, False, False),
    ]
 
mint_account.public_key

7MczJZrMTDB5eUojgdRy7MVpn2XH1v7WkcrcHfLZmJaH

In [50]:
    
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([0])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/FWDLEtjgLrQTBUK5Rab2jHWGGd3HbmBRXR2m76aL7PoMoha1zmbS57tPBmMMjiteidCQLQKM5pJAsGTsSQFbfuT?cluster=devnet'

In [51]:
time.sleep(2)
accountSchema = CStruct(
    "vault_total" / F32,
    "counter" / U32,
)

(sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
print(f"PDA: {sales_pda, sales_nonce}")

getAccountInfo(client,sales_pda)

PDA: (HwuqEf7uH9AevMoAvW427pKsNG2dk5ZMTcwRCgyn5e77, 254)


Container(vault_total=26.322830200195312, counter=15)

In [52]:
accountSchema = CStruct(
    "date_created" / U32,
    "unlockable_date" / U32,
    "numeration" /U32,
    "rarity" / U8,
    "level" / U8,
    "xp" / U32,
)


(locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
print(f"PDA: {avatar_data_pda, avatar_pda_nonce}")

getAccountInfo(client,avatar_data_pda)

PDA: (JCUCiqiiK9KDFWfaG39dAn4d9zS79wznTMKbrM9fkj3e, 252)


Container(date_created=1646396971, unlockable_date=1646396971, numeration=14, rarity=1, level=0, xp=0)

In [53]:
time.sleep(100)

new_accounts = [
    mint_account_meta,
    associated_account_meta,
    spl_program_meta,
    mint_authority_meta,
    avatar_data_pda_meta,
    sysvar_clock_meta,

    spl_program_meta
]

transaction = Transaction()
transaction.add(TransactionInstruction(
    new_accounts,
    program_id,
    bytes([1])
))
f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/4Mk7JqMiByxe8NYqFyqooLfCU2SXRju9yMxyumuVhn4QdoJN1mpbknaQT63SsMQq29BgZLMtsvjBbMkbYxELS6ke?cluster=devnet'